<h1 style="color: cyan; text-align: center;">Projet de Natural Language Processing : Analyse de sentiments des films de la plateforme TMDB</h1>

# Introduction

Ce document présente les codes Python appliqués dans le cadre du projet de traitement automatique du langage. Il présente les différentes étapes des démarches effectuées en les décrivant succintement. Pour plus de détail concernant ce travail, son objectif et sa réalisation, veuillez vous référer au document **demarche_projet.pdf** fourni avec l'ensemble des documents au moment du rendu.

## <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python fournissant aux utilisateurs des recommandations de films à regarder en priorité. Ces recommandations ne sont pas basées sur les moyens mis par les sociétés de production pour en faire la publicité mais uniquement sur le ressenti des utilisateurs. Cette approche vise à recommander films, qui pourraient être moins mis en avant dans les catalogues, pour donner de nouvelles idées au utilisateurs lorsqu'ils ne savent pas quoi regarder.

Les notes affectées aux films, pour les classer en ordre de priorité à voir, seront calculées à partir d'une analyse de sentiments des avis déposés par les utilisateurs du site https://www.themoviedb.org. Une note par avis sera calculée puis une aggrégation sera effectuée pour créer un unique indicateur par film.


<font color="red">L'objectif à la fin sera aussi de comparer nos notes déterminées sur les avis à celles déposées par les utilisateurs si il y en a eu. En effet, il arrive que les utilisateurs déposent pour un film un avis écrit et une note sur 5.

Ces avis pourraient en quelques sortes nous servir de base de test pour vérifier la fiabilité de nos analyses.
</font>

# Imports des librairies

<font color="red">Le package imdb est à importer via le code ci-dessous :</font>

```bash
pip install git+https://github.com/santhoshse7en/imdb
```

In [ ]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd
from pprint import pprint
from collections import Counter
from nltk.corpus import stopwords
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask import Flask, render_template, request

# Collecte et stockage des données

Plusieurs techniques ont été explorées pour construire notre base de données *(voir rapport pdf pour plus d'explications)*:

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

La méthode que nous avons retenue est celle à partir de l'API de IMDb. 

Avant de travailler avec l'API, il est nécessaire de créer une clé d'accès à l'API du site de themoviedb. Celle que nous utilisons est stockée dans le fichier *clefAPI.json*.

- Appel de la clé API de TMDB

In [ ]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

- Collecte des données

Nous commençons par nous restreindre à une certaine période pour les dates de sorties de film afin de ne pas avoir à traiter l'ensemble de la base de données. Cela nous évitera un temps de traitement trop important.

De plus, on on ne sélectionne que les films présents sur le site qui ont au minimum un avis déposé par un utilisateur.

In [ ]:
def get_movies_between_dates(start_date, end_date, api_key): # trouve les films sortis entre 2 dates
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            total_pages = data['total_pages']
            movies.extend([(movie['id'], movie['title']) for movie in data['results']])
            page += 1
        else:
            print("Erreur lors de la requête :", response.status_code)
            break
    return movies

def get_movie_reviews(movie_id, api_key): # retourne les avis d'un film
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['results']
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
        return None

def filter_movies_with_reviews(movies, api_key): # filtre les films qui ont des avis
    movies_with_reviews = []
    for movie_id, movie_title in movies:
        reviews = get_movie_reviews(movie_id, api_key)
        if reviews and len(reviews) > 0:
            movies_with_reviews.append((movie_id, movie_title))
    return movies_with_reviews

In [ ]:
start_date = "2022-01-01"
end_date = "2022-03-31"
print(f"Recherche des films sortis entre le {start_date} et le {end_date}")

movies = get_movies_between_dates(start_date, end_date, api_key)
print(f"Nombre de films trouvés : {len(movies)}")

movies_with_reviews = filter_movies_with_reviews(movies, api_key)
print(f"Nombre de films avec des avis de {start_date} à {end_date} : {len(movies_with_reviews)}")

## Structuration des données

Maintenant que nous disposons des données brutes pour les films ayant un ou des avis sur une certaine période, il est nécessaire de structurer les données afin de faciliter leur utilisation ensuite.

In [ ]:
# Fonction qui etourne les informations sur un film et ses avis :

def get_movie_details_and_reviews(movie_id, api_key): 
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    movie_details = response.json() if response.status_code == 200 else {}
    reviews_url = f"{base_url}{movie_id}/reviews?api_key={api_key}"
    reviews_response = requests.get(reviews_url)
    reviews_data = reviews_response.json() if reviews_response.status_code == 200 else {}
    return movie_details, reviews_data

# print(get_movie_details_and_reviews('tt0111161', api_key))

On applique cette fonction à l'ensemble des films qui rentrent dans nos conditions.

In [ ]:
movies_details = {}

for movie_id, movie_title in movies_with_reviews:
    movie_details, reviews_data = get_movie_details_and_reviews(movie_id, api_key)
    movies_details[movie_id] = {
        'title': movie_title,
        'details': movie_details,
        'reviews': reviews_data
    }

Pour éviter de relancer à chaque fois les exécutions, dont les requêtes qui peuvent être assez longues, nous stockons la base créée dans un fichier *.json*. Il suffit simplement d'accéder à ce fichier par la suite sans faire de requêtes.

<font color="red">Revoir le nom du fichier</font>

In [ ]:
# with open('movies_details_3mois.json', 'w') as json_file: 
#    json.dump(movies_details, json_file, indent=4)

# Présentation des données

Avant de se lancer dans l'analyse de sentiments, nous allons d'abord présenter les données dont nous disposons.


# <font color="red">Tout remettre au propre: on relie à chaque fois le fichier json: non</font>

In [ ]:
with open('movies_details_3mois.json', 'r') as json_file:
    movies_details = json.load(json_file)

pprint(movies_details)

In [ ]:
num_films = len(movies_details)
print("Nombres de films:", num_films)

try:
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)
    reviews_count = {}
    for movie in movies_details.values():
        num_reviews = len(movie['reviews']['results'])
        movie_id = movie['reviews']['id']
        reviews_count[movie_id] = num_reviews
    for movie_id, count in reviews_count.items():
        print(f"Film ID {movie_id}: Nombre d'avis = {count}")
except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Regardons par exemple la détail des informations pour un film; ici, le film dont l'ID est **646385**.

In [ ]:
fichier_json = 'movies_details_3mois.json'
movie_id_to_check = 646385

try:
    with open(fichier_json, 'r') as file:
        movies_details = json.load(file)

    selected_movie = movies_details[str(movie_id_to_check)]
    reviews = selected_movie['reviews']['results']
    
    for review in reviews:
        print(f"Auteur : {review['author']}")
        print(f"Rating : {review['author_details']['rating']}")
        print(f"Contenu : {review['content']}\n")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

<font color="red">Rajouter quelques stats globales</font>

# Traitement des données

Avant d'analyser le sens des avis laissés par les utilisateurs, il est nécessaire d'effectuer certeins traitements.

- On va d'abord créer une fonction clean_avis qui va nous permettre de nettoyer les commentaires récupérés sur TMDB. On va supprimer les caractères spéciaux, les mots vides et on va mettre tous les mots en minuscule. Pour cela on va s'aider de la librairie nltk et de son module stopwords et de la librairie re.

- Ensuite on va construire une fonction qui va nous permettre de compter le nombre d'occurence de chaque mot dans les commentaires. On utilisera notamment la fonction Counter de la librairie collections.

<font color="red">PENSER A RETIRER LES MOTS INUTILES, IL Y EN A ENCORE APRES LE CLEAN</font>


In [ ]:
stop_words = set(stopwords.words('english'))

def clean_avis(text):
    mots_a_retirer = ['film', 'movie']
    all_stopwords = stop_words.union(set(mots_a_retirer))
    all_stopwords = list(stop_words) + mots_a_retirer
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    words = [word for word in text.split() if word not in all_stopwords]
    return words

def analyse_word_freq(movie_reviews):
    all_words = []
    for review in movie_reviews:
        all_words.extend(clean_avis(review['content']))
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(10)
    return most_common_words

word_freq_film = {}

for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    most_common_words = analyse_word_freq(reviews)
    word_freq_film[movie_id] = most_common_words

for movie_id, frequencies in word_freq_film.items():
    print(f"{movies_details[movie_id]['title']}\nMots les plus fréquents {frequencies}\n")

# Mise en place des éléments calculés pour présenter les films

<font color="red">L'objectif de cette partie est de réaliser les calculs d'indicateurs et d'autres contenus qui seront affichés sur les pages individuelles des films.</font>

## WordClouds

Pour commencer, nous réalisons des WordClouds pour que l'utilisateur puisse, en cliquant sur un film, disposer des thèmes principaux qui sont évoqués lorsque l'on parle d'un film.

In [ ]:
def create_wordcloud(text, movie_id):
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stop_words, 
                min_font_size = 10).generate(' '.join(text))
    wordcloud.to_file(f"static/wordclouds/{movie_id}.png")
    
for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    review_text = [review['content'] for review in reviews]
    cleaned_text = []
    for text in review_text:
        cleaned_text.extend(clean_avis(text))
    create_wordcloud(cleaned_text, movie_id)

## Mise en place des informations par catégorie de film

L'objectif de l'application est de présenter une liste de film, ordonnée par ordre de priorité, à voir. Nous avons fait le choix de demander à l'utilisateur de renseigner un genre dans l'application afin de séparer les films en fonction de ses envies.

- Pour voir les genres présents dans notre base de données :

In [ ]:
def extract_unique_genres(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    unique_genres = set()
    for movie in movies_details.values():
        genres = movie.get('details', {}).get('genres', [])
        for genre in genres:
            unique_genres.add(genre['name'])

    return unique_genres

genres_uniques = extract_unique_genres('movies_details_3mois.json')
print("Genres uniques :", genres_uniques)

## Analyse des sentiments des avis

Afin de réaliser un classement des films à voir, il est nécessaire de résumer chaque film par un indicateur. Celui que nous avons choisi est la moyenne de l'ensemble des notes déterminées à partir des avis (puisque chaque film de la base possède au moins un avis).

In [ ]:
def analyse_sentiment(text): # fonction d'analyse de sentiments
    return TextBlob(text).sentiment.polarity

def calculate_movie_polarity(movie_reviews): # calcul la polarité moyenne de tous les avis d'un film
    total_polarity = 0
    num_reviews = len(movie_reviews)

    for review in movie_reviews:
        total_polarity += analyse_sentiment(review['content'])

    return total_polarity / num_reviews if num_reviews > 0 else 0

def genre_top_movies(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    genre_movies = {genre: [] for genre in genres_uniques}

    for movie_id, movie in movies_details.items():
        movie_polarity = calculate_movie_polarity(movie['reviews']['results'])
        movie_genres = [genre['name'] for genre in movie['details']['genres']]

        for genre in movie_genres:
            if genre in genre_movies:
                genre_movies[genre].append((movie_id, movie['title'], movie_polarity))

    top_movies_per_genre = {}

    for genre, movies in genre_movies.items():
        top_movies = sorted(movies, key=lambda x: x[2], reverse=True)[:3]
        top_movies_per_genre[genre] = top_movies

    return top_movies_per_genre
    # renvoie un dictionnaire où chaque clé est un genre et chaque valeur est une liste de tuples 
    # contenant l'ID, le titre et la polarité moyenne des 3 meilleurs films de ce genre

top_movies = genre_top_movies('movies_details_3mois.json')
print(top_movies)

# Création de l'application finale

NOus finissons par créer un support visuel pour présenter nos résultats.

In [ ]:
app = Flask(__name__)

def polarity_to_stars(polarity):
    return round(polarity * 10)

app.jinja_env.globals.update(polarity_to_stars=polarity_to_stars)

@app.route('/', methods=['GET', 'POST'])
def index():
    genres = list(top_movies.keys())
    selected_genre = request.form.get('genre')
    top_movies_genre = top_movies.get(selected_genre, []) if selected_genre else []

    movies_with_images = []
    for movie_id, movie_title, polarity in top_movies_genre:
        image_url = movies_details[movie_id].get("image_url", "")
        movies_with_images.append((movie_id, movie_title, polarity, image_url))

    return render_template('index2.html', genres=genres, top_movies_genre=movies_with_images, selected_genre=selected_genre)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)